<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning and watson openscale for Integrated Experience

This notebook should be run in a Watson Studio project, using **IBM Runtime 23.1 on Python 3.10 XS** runtime environment. **If you are viewing this in Watson Studio and do not see the required runtime env in the upper right corner of your screen, please update the runtime now.** It requires service credentials for the following Cloud services:
  * Watson OpenScale 
  * Watson Machine Learning 
  * Cloud Object Storage
  
If you have a paid Cloud account, you may also provision a **Databases for PostgreSQL** or **Db2 Warehouse** service to take full advantage of integration with Watson Studio. If you choose not to provision this paid service, you can use the free internal PostgreSQL storage with OpenScale, but will not be able to configure continuous learning for your model.

The notebook will train, create and deploy a German Credit Risk model, configure OpenScale to monitor that deployment.

## Contents

- [Installing required packages used by this notebook](#Installing-required-packages-used-by-this-notebook)
- [Provision services and configure credentials](#Provision-services-and-configure-credentials)
- [Configuring dataset and store dataset it Cloud Object Storage](#Configuring-dataset-and-store-it-Cloud-Object-Storage)
- [Creating watsonx_ai client ](#Creating-watsonx_ai-client)
- [Creating watson openscale client ](#Creating-watson-openscale-client)
- [Creating and training wml model](#Creating-and-training-wml-model)
- [Creating configuration for watson openscale](#Creating-configuration-for-watson-openscale)
- [Storing wml model in pre_production space](#Storing-wml-model-in-pre_production-space)
- [OpenScale configuration using Intergrated Expirence for a pre_production model](#OpenScale-configuration-using-Intergrated-Expirence-for-a-pre_production-model)
- [Storing wml model in production space](#Storing-wml-model-in-production-space)
- [OpenScale configuration using Intergrated Expirence for a production model](#OpenScale-configuration-using-Intergrated-Expirence-for-a-production-model)
- [Dashboard url for both production and pre-production model](#Dashboard-url-of-both-production-and-pre-production-model)

# Installing required packages used by this notebook

In [1]:
# If you are executing this notebook in non IBM Watson Studio env then uncomment the pip install statements
# !pip install --upgrade scikit-learn==1.3 --no-cache | tail -n 1
# !pip install --upgrade joblib==1.3 --no-cache | tail -n 1
# !pip install --upgrade SciPy==1.14.0  --no-cache | tail -n 1
# !pip install --upgrade ibm-db --no-cache | tail -n 1
# !pip install ibm_cloud_sdk_core --no-cache | tail -n 1

!pip install --upgrade retrying --no-cache | tail -n 1  
!pip install --upgrade ibm-watsonx-ai | tail -n 1 
# !pip install --upgrade ibm-watson-openscale>=3.0.42 --no-cache | tail -n 1  

<h3> Make sure you have ibm-watson-openscale package version greater than or Equal 3.0.42 </h3>

In [2]:
!pip list | grep ibm-watson-openscale

# Provision services and configure credentials
If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key, generate an IAM token using that key and paste it below.

**NOTE:** You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using console:
```
bx login --sso
bx iam api-key-create 'my_key'
```
__Please replace <'EDIT THIS'> to your values *__

In [3]:
CLOUD_API_KEY = "<EDIT THIS>"
IAM_URL = "https://iam.cloud.ibm.com/oidc/token"

## WML credentials

In [4]:
WML_CREDENTIALS = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": CLOUD_API_KEY
}

This tutorial can use Databases for PostgreSQL, Db2 Warehouse, or a free internal verison of PostgreSQL to create a datamart for OpenScale.

If you have previously configured OpenScale, it will use your existing datamart, and not interfere with any models you are currently monitoring. Do not update the cell below.

If you do not have a paid Cloud account or would prefer not to provision this paid service, you may use the free internal PostgreSQL service with OpenScale. Do not update the cell below.

To provision a new instance of Db2 Warehouse, locate [Db2 Warehouse in the Cloud catalog](https://cloud.ibm.com/catalog/services/db2-warehouse), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Db2 Warehouse credentials into the cell below.

In [5]:
# Configure DB_CREDENTIALS to create datamart with an external database
# DB_CREDENTIALS = {
#     "database_type"='postgresql' # Mention db2 as database_type if Database is DB2
#     "hostname":"",
#     "username":"",
#     "password":"",
#     "database":"",
#     "port":31876,
#     "ssl":False,
#     "sslmode":None,
#     "certificate_base64":None
#     }

DB_CREDENTIALS = None
SCHEMA_NAME = None

__If you previously configured OpenScale to use the free internal version of PostgreSQL, you can switch to a new datamart using a paid database service.__ If you would like to delete the internal PostgreSQL configuration and create a new one using service credentials supplied in the cell above, set the __KEEP_MY_INTERNAL_POSTGRES__ variable below to __False__ below. In this case, the notebook will remove your existing internal PostgreSQL datamart and create a new one with the supplied credentials. __*NO DATA MIGRATION WILL OCCUR.*__ This NoteBook configured OpenScale to use the free internal version of PostgreSQL*

In [6]:
KEEP_MY_INTERNAL_POSTGRES = True

In next cells, you will need to paste some credentials to Cloud Object Storage. If you haven't worked with COS yet please visit [getting started with COS tutorial](https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-getting-started). 
You can find `COS_API_KEY_ID` and `COS_RESOURCE_CRN` variables in **_Service Credentials_** in menu of your COS instance. Used COS Service Credentials must be created with _Role_ parameter set as Writer. Later training data file will be loaded to the bucket of your instance and used as training refecence in subsription.  
`COS_ENDPOINT` variable can be found in **_Endpoint_** field of the menu.

In [7]:
COS_API_KEY_ID = "<EDIT THIS>"
COS_RESOURCE_CRN = "<EDIT THIS>"  # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "<EDIT THIS>" # "https://s3.us-south.cloud-object-storage.appdomain.cloud"

In [8]:
BUCKET_NAME = "<EDIT THIS>" # example: "credit-risk-training-data"

# Configuring dataset and store it Cloud Object Storage
## Load the training data from github

In [9]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_data_biased_training.csv -O german_credit_data_biased_training.csv

--2024-10-30 11:44:17--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_data_biased_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689622 (673K) [text/plain]
Saving to: ‘german_credit_data_biased_training.csv’

german_credit_data_ 100%[===================>] 673.46K  --.-KB/s    in 0.07s   

2024-10-30 11:44:18 (9.45 MB/s) - ‘german_credit_data_biased_training.csv’ saved [689622/689622]



In [10]:
import numpy as np
import pandas as pd

training_data_file_name = "german_credit_data_biased_training.csv"
data_df = pd.read_csv(training_data_file_name)
print(data_df.head())
print('Columns: ', list(data_df.columns))
print('Number of columns: ', len(data_df.columns))

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


Columns:  ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'Risk']
Number of columns:  21


#### As you can see, the data contains twenty one fields. ```Risk``` field is the one you would like to predict using feedback data.

In [11]:
print('Number of records: ', data_df.Risk.count())
target_count = data_df.groupby('Risk')['Risk'].count()
print('Target count: ', target_count)

Number of records:  5000
Target count:  Risk
No Risk    3330
Risk       1670
Name: Risk, dtype: int64


## Save training data to Cloud Object Storage

In [12]:
import ibm_boto3
from ibm_botocore.client import Config

cos_client = ibm_boto3.resource("s3",
                                ibm_api_key_id=COS_API_KEY_ID,
                                ibm_service_instance_id=COS_RESOURCE_CRN,
                                ibm_auth_endpoint=IAM_URL,
                                config=Config(signature_version="oauth"),
                                endpoint_url=COS_ENDPOINT
                                )
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

# Creating watsonx_ai client

In [13]:
from ibm_watsonx_ai import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.1.20'

# Creating watson openscale client

In [14]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_openscale import *

authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
pre_prod_ai_client = APIClient(authenticator=authenticator)
prod_ai_client = APIClient(authenticator=authenticator)

# Creating and training wml model
## Importing required libraries and creating model

In [15]:
# You will start with importing required libraries
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance

In [16]:
# Splitting the data into train and test
train_data, test_data = train_test_split(data_df, test_size=0.2)

# Preparing the pipeline
features_idx = np.s_[0:-1]
first_record_idx = np.s_[0]

# In this step, you will encode target column labels into numeric values. You can use `inverse_transform` to decode numeric predictions into labels.
string_fields = [type(fld) is str for fld in train_data.iloc[first_record_idx, features_idx]]
ct = ColumnTransformer([("ohe", OneHotEncoder(), list(np.array(train_data.columns)[features_idx][string_fields]))])
clf_linear = SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, tol=1e-5)
pipeline_linear = Pipeline([('ct', ct), ('clf_linear', clf_linear)])

# Train a model
print("Training a model")
risk_model = pipeline_linear.fit(train_data.drop('Risk', axis=1), train_data.Risk)

# Evaluate the model
print("Evaluating the model")
predictions = risk_model.predict(test_data.drop('Risk', axis=1))
indexed_preds = [0 if prediction == 'No Risk' else 1 for prediction in predictions]
real_observations = test_data.Risk.replace('Risk', 1)
real_observations = real_observations.replace('No Risk', 0).values
auc = roc_auc_score(real_observations, indexed_preds)
print(auc)

Training a model
Evaluating the model
0.7363105115071062


## Finding feature importance for a model deployed in Watson Machine Learning (WML) to understand feature impact on predictions.

In [17]:
# This is done because we will deduce the important and most important features to the model. This will help in narrowing down the analysis of Drift v2 evaluation in the UI
per_res = permutation_importance(risk_model, train_data.drop('Risk', axis=1), train_data.Risk, random_state=0)
per_res = np.mean(np.abs(per_res.importances), axis=1)
fields = [col for col in train_data.columns if col != "Risk"]
feature_importance = dict(zip(fields, per_res))
print(feature_importance)

{'CheckingStatus': 0.012600000000000056, 'LoanDuration': 0.0, 'CreditHistory': 0.002049999999999974, 'LoanPurpose': 0.002800000000000047, 'LoanAmount': 0.0, 'ExistingSavings': 0.0020500000000000405, 'EmploymentDuration': 0.02185000000000008, 'InstallmentPercent': 0.0, 'Sex': 0.006350000000000034, 'OthersOnLoan': 0.0025500000000000522, 'CurrentResidenceDuration': 0.0, 'OwnsProperty': 0.01700000000000006, 'Age': 0.0, 'InstallmentPlans': 0.0024000000000000245, 'Housing': 0.001350000000000029, 'ExistingCreditsCount': 0.0, 'Job': 0.0025000000000000135, 'Dependents': 0.0, 'Telephone': 0.0021999999999999797, 'ForeignWorker': 0.0006999999999999895}


# Creating configuration for watson openscale
## Database configuration

In [18]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import DatabaseConfigurationRequest, \
    PrimaryStorageCredentialsLong, LocationSchemaName

db_credentials = None
location = None

# Below Code is set up External Datamart Configuration
if DB_CREDENTIALS is not None:
    if SCHEMA_NAME is not None:
        location = LocationSchemaName(
            schema_name=SCHEMA_NAME
        )
    if DB_CREDENTIALS["ssl"] is True and DB_CREDENTIALS["sslmode"] is None and DB_CREDENTIALS[
        "certificate_base64"] is None:
        raise Exception("If SSL is enabled, SSL mode and SSL certificate must be provided ")

    db_credentials = DatabaseConfigurationRequest(
        database_type=DB_CREDENTIALS["database_type"],  
        credentials=PrimaryStorageCredentialsLong(
            hostname=DB_CREDENTIALS["hostname"],
            username=DB_CREDENTIALS["username"],
            password=DB_CREDENTIALS["password"],
            db=DB_CREDENTIALS["database"],
            port=DB_CREDENTIALS["port"],
            ssl=DB_CREDENTIALS["ssl"],
            sslmode=DB_CREDENTIALS["sslmode"],
            certificate_base64=DB_CREDENTIALS["certificate_base64"]
        ),
        location=location)

## Configure label column

In [19]:
label_column = "Risk"

## Configure fields

In [20]:
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
          "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
          "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
          "Telephone", "ForeignWorker"]

## Configure feature columns values

In [21]:
feature_columns = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
                   "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
                   "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
                   "Telephone", "ForeignWorker"]

## Configure categorical column values

In [22]:
categorical_columns = ["CheckingStatus", "CreditHistory", "LoanPurpose", "ExistingSavings", "EmploymentDuration", "Sex",
                       "OthersOnLoan", "OwnsProperty", "InstallmentPlans", "Housing", "Job", "Telephone",
                       "ForeignWorker"]

## Configure COS details

In [23]:
cos_training_data = {
    "type": "cos",
    "connection": {
        "resource_instance_id": COS_RESOURCE_CRN,
        "url": COS_ENDPOINT,
        "api_key": COS_API_KEY_ID,
        "iam_url": IAM_URL
    },
    "location": {
        "bucket": BUCKET_NAME,
        "file_name": training_data_file_name
    }
}

## Configure quality monitor details

In [24]:
quality_config = {
    "min_records": 50,
    "max_records": 1000,
    "threshold": [
        {
            "metric_id": "area_under_roc",
            "type": "lower_limit",
            "value": .80
        }
    ]
}

## Configure fairness monitor details

In [25]:
fairness_config = {
    "favorable_classes": ["No Risk"],
    "unfavorable_classes": ["Risk"],
    "min_records": 50,
    "features": [
        {"name": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"name": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }]
}

## Configure drift_v2 monitor details

In [26]:
drift_v2_config = {
    "min_records": 50,
    "max_records": 1000,
    "train_archive": True,
    "fields": fields,
    "importance":feature_importance,
    "thresholds": [
        {
            "metric_id": "confidence_drift_score",
            "type": "lower_limit",
            "value": 0.8
        }
    ]
}

# Storing model and configure watson openscale in pre_production WML instance
## Storing wml model in pre_production space

In [27]:
PRE_PROD_MODEL_NAME = "Scikit German Risk Model WML V4 pre-production Model"
PRE_PROD_DEPLOYMENT_NAME = "Scikit German Risk Deployment WML V4 pre-production Model"

## Listing all the available spaces

In [28]:
wml_client.spaces.list(limit=10)

,ID,NAME,CREATED
0,d86b598d-a69d-4fdc-89a6-3c6c5cd7a428,v2-e2e-test-space,2024-10-23T06:25:57.849Z
1,d8b93cb6-2caa-49f4-b08e-6a6b6521adee,Prod_space,2024-10-22T08:52:41.226Z
2,fc5257ff-f3a2-4fe9-a8d8-0f759e1f8007,Pre_prod_space,2024-10-22T08:38:44.167Z
3,d5613787-9178-435b-a8f8-590b9a9c47f2,sampl123,2024-10-11T06:16:10.258Z
4,bc592530-1cf0-4e56-9fab-70e0924f6907,prompt_evaluvation_s,2024-09-17T15:39:39.598Z
5,00883863-b2ef-4b90-b797-f5917a6618bc,Notebook,2024-07-16T09:30:02.576Z
6,523c8856-6d1e-49aa-b371-5dab42cd322c,sdk-test-space,2024-06-27T09:19:53.318Z


## Setting up default space Id

In [29]:
WML_PRE_PROD_SPACE_ID = "<EDIT THIS>"
wml_client.set.default_space(WML_PRE_PROD_SPACE_ID)

'SUCCESS'

## Publish the model in wml
### Remove existing model and deployment from default space

In [30]:
print("Removing existing model and deployment....")
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == PRE_PROD_DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)
        display(wml_client.repository.list_models())

Removing existing model and deployment....


In [31]:
datasource_type = wml_client.connections.get_datasource_type_uid_by_name('bluemixcloudobjectstorage')
conn_meta_props = {
    wml_client.connections.ConfigurationMetaNames.NAME: "Connection My COS ",
    wml_client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: datasource_type,
    wml_client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection to my COS",
    wml_client.connections.ConfigurationMetaNames.PROPERTIES: {
        'bucket': BUCKET_NAME,
        'api_key': COS_API_KEY_ID,
        'resource_instance_id': COS_RESOURCE_CRN,
        'iam_url': IAM_URL,
        'url': COS_ENDPOINT
    }
}

conn_details = wml_client.connections.create(meta_props=conn_meta_props)
connection_id = wml_client.connections.get_uid(conn_details)

training_data_references = [
    {
        "id": "German Credit Risk",
        "type": "connection_asset",
        "connection": {
            "id": connection_id,
            "href": "/v2/connections/" + connection_id + "?space_id=" + WML_PRE_PROD_SPACE_ID

        },
        "location": {
            "bucket": BUCKET_NAME,
            "file_name": training_data_file_name
        }
    }
]

Creating connections...
SUCCESS


In [32]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
print("Software Specification ID: {}".format(software_spec_uid))

model_props = {
    wml_client._models.ConfigurationMetaNames.NAME: "{}".format(PRE_PROD_MODEL_NAME),
    wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.3",
    wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client._models.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
    wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "Risk",
}

Software Specification ID: 45f12dfe-aa78-5b8d-9f38-0ee223c47309


## Storing model in wml

In [33]:
print("Storing model....")
published_model_details = wml_client.repository.store_model(model=risk_model, meta_props=model_props,
                                                            training_data=data_df.drop(["Risk"], axis=1),
                                                            training_target=data_df.Risk)
pre_prod_model_uid = wml_client.repository.get_model_id(published_model_details)
print("Done \nModel ID: {}".format(pre_prod_model_uid))

Storing model....
Done 
Model ID: 62ae27d6-7df8-4d76-b507-aee9121459af


## Deploy model in wml

In [34]:
print("Deploying model....")
deployment_details = wml_client.deployments.create(
    pre_prod_model_uid,
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(PRE_PROD_DEPLOYMENT_NAME),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
pre_prod_scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
pre_prod_deployment_uid = wml_client.deployments.get_uid(deployment_details)
print("Scoring URL: {} \nModel id: {} \nDeployment id: {}".format(pre_prod_scoring_url, pre_prod_model_uid,
                                                                  pre_prod_deployment_uid))

Deploying model....


######################################################################################

Synchronous deployment creation for id: '62ae27d6-7df8-4d76-b507-aee9121459af' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.

ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='c44269c8-7e2c-4ab5-acd8-1ea36a046b28'
-----------------------------------------------------------------------------------------------


Scoring URL: https://us-south.ml.cloud.ibm.com/ml/v4/deployments/c44269c8-7e2c-4ab5-acd8-1ea36a046b28/predictions 
Model id: 62ae27d6-7df8-4d76-b507-aee9121459af 
Deployment id: c44269c8-7e2c-4ab5-acd8-1ea36a046b28


## Scoring the wml model

In [35]:
print("Scoring model....")
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
          "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
          "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
          "Telephone", "ForeignWorker"]
values = [
    ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
     "savings_insurance", 46, "none", "own", 2, "skilled", 1, "none", "yes"],
    ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
     4, "savings_insurance", 36, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
]
scoring_payload = {"input_data": [{"fields": fields, "values": values}]}
predictions = wml_client.deployments.score(pre_prod_deployment_uid, scoring_payload)
print(predictions)

Scoring model....
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [['Risk', [0.4882215859973983, 0.5117784140026017]], ['No Risk', [0.6142450995034986, 0.3857549004965014]]]}]}


# OpenScale configuration using Intergrated Expirence for a pre_production model
### Setting up operational_space_id to either "pre_production" or "production" based on your Watson Machine Learning (WML) model environment.

In [36]:
operational_space_id = "pre_production"

## Below cell create object of Watson Machine Learning Model Configuration details used in evaluate function

In [37]:
from ibm_watson_openscale.base_classes.models.engine_models.wml import WatsonMachineLearningModel

model = WatsonMachineLearningModel(model_uid=pre_prod_model_uid,
                                   deployment_uid=pre_prod_deployment_uid,
                                   space_uid=WML_PRE_PROD_SPACE_ID,
                                   label_column=label_column,
                                   training_data_reference=cos_training_data,
                                   test_data_df=data_df,
                                   favorable_classes=['No Risk'],
                                   unfavorable_classes=['Risk'],
                                   quality_config=quality_config,
                                   fairness_config=fairness_config,
                                   drift_v2_config=drift_v2_config,
                                   prediction_column="prediction",
                                   probability_column="probability",
                                   feature_columns=feature_columns,
                                   categorical_columns=categorical_columns
                                   )

## The following cell execute evaluate function which automatically creates datamart , service provider , subscription and enables monitors - fairness , quality, drift v2. 

In [38]:
result = pre_prod_ai_client.model.evaluate(model, engine_credentials=WML_CREDENTIALS, db_credentials=db_credentials,
                                           operational_space_id=operational_space_id)
subscription_id = result[1]

Setting up Datamart...
Using existing datamart
Setting up Service Binding...
Using existing binding 6ba953e5-a0c4-4008-9a72-3eef7316350a
Validating model...
Existence check for deployment finished. Deployment is Scikit German Risk Model WML V4 pre-production Model
Existence check for Model finished.
Finished reading training data...
Finished inferring model details
Creating Subscription...



 Waiting for end of deleting subscription ec994c09-59ea-46e6-97f2-fdba735a1b9f 




finished

---------------------------------------------
 Successfully finished deleting subscription 
---------------------------------------------


Deleted existing subscription for Scikit German Risk Model WML V4 pre-production Model



 Waiting for end of adding subscription af373042-e600-4c0a-b2fd-04981cfdb0e0 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------


Subscription created successfully. Id af373042-e600-4c0a

## To evaluate monitors again, run the below cell

In [ ]:
# pre_prod_ai_client.model.run_evaluation()

## Get monitor instance details for the configured monitors
 The following cell will collect monitor_instance_id details for particular subscription_Id from ibm_watson_opscale SDK.

In [40]:
monitor_instance_ids = {}
monitor_instance_list = pre_prod_ai_client.monitor_instances.list(target_target_id=subscription_id).result
for instance in monitor_instance_list.monitor_instances:
    monitor_instance_ids.update({instance.entity.monitor_definition_id: instance.metadata.id})
monitor_instance_ids

{'quality': 'f322fff4-2ae1-4fab-a894-0677a82a61ae',
 'fairness': '966c5191-8ce4-4675-854e-8c2367aabb52',
 'mrm': 'a944cb6c-ac17-4be6-ae16-5aa5e1ef6bf8',
 'drift_v2': '0ba6c52e-402f-4402-ba19-4f67ec052f26',
 'explainability': '1258a533-154e-42da-b576-04f6f66b48b8',
 'performance': 'faafabea-86a1-4444-96eb-8c32602ddb1e'}

## Fairness monitor details

In [41]:
pre_prod_ai_client.monitor_instances.show_metrics(monitor_instance_id=monitor_instance_ids['fairness'])

2024-10-30 06:27:35.763033+00:00,fairness_value,f00ffe2c-da35-4af2-883e-4d5c6d077d4b,90.0,95.0,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,966c5191-8ce4-4675-854e-8c2367aabb52,8f436fec-c728-4c31-8584-fef4e1315643,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:27:35.763033+00:00,fairness_value,f00ffe2c-da35-4af2-883e-4d5c6d077d4b,100.0,95.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,966c5191-8ce4-4675-854e-8c2367aabb52,8f436fec-c728-4c31-8584-fef4e1315643,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:21:52.999022+00:00,fairness_value,dbf5c7fc-64b7-48e8-a8d0-70b665eddcf8,90.0,95.0,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,966c5191-8ce4-4675-854e-8c2367aabb52,16a80f20-f583-434a-a63b-977f32142be4,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:21:52.999022+00:00,fairness_value,dbf5c7fc-64b7-48e8-a8d0-70b665eddcf8,100.0,95.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,966c5191-8ce4-4675-854e-8c2367aabb52,16a80f20-f583-434a-a63b-977f32142be4,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0


## Quality monitor details

In [42]:
pre_prod_ai_client.monitor_instances.show_metrics(monitor_instance_id=monitor_instance_ids['quality'])

2024-10-30 06:27:33.643000+00:00,true_positive_rate,03588d2c-8f47-4a89-8e42-b13d9e005621,0.6685714285714286,None,None,['model_type:original'],quality,f322fff4-2ae1-4fab-a894-0677a82a61ae,f7136e5b-a611-428b-8707-0b3a55dace0e,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:27:33.643000+00:00,area_under_roc,03588d2c-8f47-4a89-8e42-b13d9e005621,0.7427472527472527,0.8,None,['model_type:original'],quality,f322fff4-2ae1-4fab-a894-0677a82a61ae,f7136e5b-a611-428b-8707-0b3a55dace0e,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:27:33.643000+00:00,precision,03588d2c-8f47-4a89-8e42-b13d9e005621,0.6628895184135978,None,None,['model_type:original'],quality,f322fff4-2ae1-4fab-a894-0677a82a61ae,f7136e5b-a611-428b-8707-0b3a55dace0e,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:27:33.643000+00:00,matthews_correlation_coefficient,03588d2c-8f47-4a89-8e42-b13d9e005621,0.4845465733937984,None,None,['model_type:original'],quality,f322fff4-2ae1-4fab-a894-0677a82a61ae,f7136e5b-a611-428b-8707-0b3a55dace0e,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:27:33.643000+00:00,f1_measure,03588d2c-8f47-4a89-8e42-b13d9e005621,0.6657183499288764,None,None,['model_type:original'],quality,f322fff4-2ae1-4fab-a894-0677a82a61ae,f7136e5b-a611-428b-8707-0b3a55dace0e,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:27:33.643000+00:00,accuracy,03588d2c-8f47-4a89-8e42-b13d9e005621,0.765,None,None,['model_type:original'],quality,f322fff4-2ae1-4fab-a894-0677a82a61ae,f7136e5b-a611-428b-8707-0b3a55dace0e,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:27:33.643000+00:00,label_skew,03588d2c-8f47-4a89-8e42-b13d9e005621,0.6289709020331526,None,None,['model_type:original'],quality,f322fff4-2ae1-4fab-a894-0677a82a61ae,f7136e5b-a611-428b-8707-0b3a55dace0e,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:27:33.643000+00:00,gini_coefficient,03588d2c-8f47-4a89-8e42-b13d9e005621,0.48549450549450546,None,None,['model_type:original'],quality,f322fff4-2ae1-4fab-a894-0677a82a61ae,f7136e5b-a611-428b-8707-0b3a55dace0e,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:27:33.643000+00:00,log_loss,03588d2c-8f47-4a89-8e42-b13d9e005621,0.45635328736659064,None,None,['model_type:original'],quality,f322fff4-2ae1-4fab-a894-0677a82a61ae,f7136e5b-a611-428b-8707-0b3a55dace0e,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:27:33.643000+00:00,false_positive_rate,03588d2c-8f47-4a89-8e42-b13d9e005621,0.18307692307692308,None,None,['model_type:original'],quality,f322fff4-2ae1-4fab-a894-0677a82a61ae,f7136e5b-a611-428b-8707-0b3a55dace0e,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0


Note: First 10 records were displayed.


## Drift_v2 monitor details

In [43]:
pre_prod_ai_client.monitor_instances.show_metrics(monitor_instance_id=monitor_instance_ids['drift_v2'])

2024-10-30 06:28:06.767386+00:00,records_processed,287a2738-6cc4-42d1-86a5-e10dcfb50cf0,1000.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0ba6c52e-402f-4402-ba19-4f67ec052f26,fd20e756-8f1a-4503-aba5-10e0641fb669,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:28:06.767386+00:00,confidence_drift_score,287a2738-6cc4-42d1-86a5-e10dcfb50cf0,0.0867,0.8,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0ba6c52e-402f-4402-ba19-4f67ec052f26,fd20e756-8f1a-4503-aba5-10e0641fb669,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:28:06.767386+00:00,records_processed,287a2738-6cc4-42d1-86a5-e10dcfb50cf0,1000.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0ba6c52e-402f-4402-ba19-4f67ec052f26,fd20e756-8f1a-4503-aba5-10e0641fb669,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:28:06.767386+00:00,confidence_drift_score,287a2738-6cc4-42d1-86a5-e10dcfb50cf0,0.0876,0.8,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0ba6c52e-402f-4402-ba19-4f67ec052f26,fd20e756-8f1a-4503-aba5-10e0641fb669,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:28:17.279686+00:00,records_processed,9c864a34-dd6d-418a-b584-27d31aac0461,1000.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0ba6c52e-402f-4402-ba19-4f67ec052f26,fd20e756-8f1a-4503-aba5-10e0641fb669,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:28:17.279686+00:00,confidence_drift_score,9c864a34-dd6d-418a-b584-27d31aac0461,0.0782,0.8,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0ba6c52e-402f-4402-ba19-4f67ec052f26,fd20e756-8f1a-4503-aba5-10e0641fb669,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:28:17.279686+00:00,records_processed,9c864a34-dd6d-418a-b584-27d31aac0461,1000.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0ba6c52e-402f-4402-ba19-4f67ec052f26,fd20e756-8f1a-4503-aba5-10e0641fb669,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:28:17.279686+00:00,confidence_drift_score,9c864a34-dd6d-418a-b584-27d31aac0461,0.0802,0.8,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0ba6c52e-402f-4402-ba19-4f67ec052f26,fd20e756-8f1a-4503-aba5-10e0641fb669,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:28:17.677174+00:00,records_processed,20228d06-f077-4b92-bd18-7531f33e8d90,1000.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,0ba6c52e-402f-4402-ba19-4f67ec052f26,fd20e756-8f1a-4503-aba5-10e0641fb669,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0
2024-10-30 06:28:17.677174+00:00,prediction_drift_score,20228d06-f077-4b92-bd18-7531f33e8d90,0.0222,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,0ba6c52e-402f-4402-ba19-4f67ec052f26,fd20e756-8f1a-4503-aba5-10e0641fb669,subscription,af373042-e600-4c0a-b2fd-04981cfdb0e0


Note: First 10 records were displayed.


## User can navigate to see an evaluated result in the UI 
### Configure dashboard url

In [44]:
pre_prod_integrated_url = pre_prod_ai_client.service_url[0:pre_prod_ai_client.service_url.index("/openscale")].replace(
    "api.aiopenscale",
    "dataplatform") + f"/ml-runtime/deployments/{pre_prod_deployment_uid}/evaluations?space_id={WML_PRE_PROD_SPACE_ID}"
print(f"DashBoard URL: \nIntegrated Experience url: {pre_prod_integrated_url}")

DashBoard URL: 
Integrated Experience url: https://dataplatform.cloud.ibm.com/ml-runtime/deployments/c44269c8-7e2c-4ab5-acd8-1ea36a046b28/evaluations?space_id=fc5257ff-f3a2-4fe9-a8d8-0f759e1f8007


#
# Promote pre-production model to production model

#### After you have reviewed the evaluation results of the PreProd and if you make the decision to promote the PreProd model to Production, the first thing you need to do is to deploy the model into a WML instance that is designated as Production instance

# Storing model and configure watson openscale in production WML instance
## Storing wml model in production space

In [45]:
PROD_MODEL_NAME = "Scikit German Risk Model WML V4 production Model"
PROD_DEPLOYMENT_NAME = "Scikit German Risk Deployment WML V4 production Model"

## Listing all the available spaces

In [46]:
wml_client.spaces.list(limit=10)

,ID,NAME,CREATED
0,d86b598d-a69d-4fdc-89a6-3c6c5cd7a428,v2-e2e-test-space,2024-10-23T06:25:57.849Z
1,d8b93cb6-2caa-49f4-b08e-6a6b6521adee,Prod_space,2024-10-22T08:52:41.226Z
2,fc5257ff-f3a2-4fe9-a8d8-0f759e1f8007,Pre_prod_space,2024-10-22T08:38:44.167Z
3,d5613787-9178-435b-a8f8-590b9a9c47f2,sampl123,2024-10-11T06:16:10.258Z
4,bc592530-1cf0-4e56-9fab-70e0924f6907,prompt_evaluvation_s,2024-09-17T15:39:39.598Z
5,00883863-b2ef-4b90-b797-f5917a6618bc,Notebook,2024-07-16T09:30:02.576Z
6,523c8856-6d1e-49aa-b371-5dab42cd322c,sdk-test-space,2024-06-27T09:19:53.318Z


In [47]:
WML_PROD_SPACE_ID = "<EDIT THIS>"
wml_client.set.default_space(WML_PROD_SPACE_ID)

'SUCCESS'

## Remove existing model and deployment from default space

In [49]:
print("Removing existing model and deployment....")
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == PROD_DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)
        display(wml_client.repository.list_models())

Removing existing model and deployment....


## Publish the model in wml

In [50]:
datasource_type = wml_client.connections.get_datasource_type_uid_by_name('bluemixcloudobjectstorage')
conn_meta_props = {
    wml_client.connections.ConfigurationMetaNames.NAME: "Connection My COS ",
    wml_client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: datasource_type,
    wml_client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection to my COS",
    wml_client.connections.ConfigurationMetaNames.PROPERTIES: {
        'bucket': BUCKET_NAME,
        'api_key': COS_API_KEY_ID,
        'resource_instance_id': COS_RESOURCE_CRN,
        'iam_url': "https://iam.ng.bluemix.net/oidc/token",
        'url': COS_ENDPOINT
    }
}

conn_details = wml_client.connections.create(meta_props=conn_meta_props)
connection_id = wml_client.connections.get_uid(conn_details)

training_data_references = [
    {
        "id": "German Credit Risk",
        "type": "connection_asset",
        "connection": {
            "id": connection_id,
            "href": "/v2/connections/" + connection_id + "?space_id=" + WML_PROD_SPACE_ID

        },
        "location": {
            "bucket": BUCKET_NAME,
            "file_name": training_data_file_name
        }
    }
]

Creating connections...
SUCCESS


In [51]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
print("Software Specification ID: {}".format(software_spec_uid))

model_props = {
    wml_client._models.ConfigurationMetaNames.NAME: "{}".format(PROD_MODEL_NAME),
    wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.3",
    wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client._models.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
    wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "Risk",
}

Software Specification ID: 45f12dfe-aa78-5b8d-9f38-0ee223c47309


## Storing model in wml

In [52]:
print("Storing model....")
published_model_details = wml_client.repository.store_model(model=risk_model, meta_props=model_props,
                                                            training_data=data_df.drop(["Risk"], axis=1),
                                                            training_target=data_df.Risk)
prod_model_uid = wml_client.repository.get_model_id(published_model_details)
print("Done \nModel ID: {}".format(prod_model_uid))

Storing model....
Done 
Model ID: e5005ee3-14dd-46d0-b313-f61689bcd9cc


## Deploy model in wml

In [53]:
print("Deploying model....")
deployment_details = wml_client.deployments.create(
    prod_model_uid,
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(PROD_DEPLOYMENT_NAME),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
prod_scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
prod_deployment_uid = wml_client.deployments.get_uid(deployment_details)
print("Scoring URL: {} \nModel id: {} \nDeployment id: {}".format(prod_scoring_url, prod_model_uid, prod_deployment_uid))

Deploying model....


######################################################################################

Synchronous deployment creation for id: 'e5005ee3-14dd-46d0-b313-f61689bcd9cc' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
.
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='30c56a1f-ad47-4de2-95b0-41ce84447f76'
-----------------------------------------------------------------------------------------------


Scoring URL: https://us-south.ml.cloud.ibm.com/ml/v4/deployments/30c56a1f-ad47-4de2-95b0-41ce84447f76/predictions 
Model id: e5005ee3-14dd-46d0-b313-f61689bcd9cc 
Deployment id: 30c56a1f-ad47-4de2-95b0-41ce84447f76


## Scoring the wml model

In [54]:
print("Scoring model....")
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
          "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
          "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
          "Telephone", "ForeignWorker"]
values = [
    ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
     "savings_insurance", 46, "none", "own", 2, "skilled", 1, "none", "yes"],
    ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
     4, "savings_insurance", 36, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
]
scoring_payload = {"input_data": [{"fields": fields, "values": values}]}
predictions = wml_client.deployments.score(prod_deployment_uid, scoring_payload)
print(predictions)

Scoring model....
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [['Risk', [0.4882215859973983, 0.5117784140026017]], ['No Risk', [0.6142450995034986, 0.3857549004965014]]]}]}


# OpenScale configuration using Intergrated Expirence for a production model

In [55]:
operational_space_id = "production"

## Below cell create object of Watson Machine Learning Model Configuration details used in evaluate function

In [56]:
from ibm_watson_openscale.base_classes.models.engine_models.wml import WatsonMachineLearningModel

model = WatsonMachineLearningModel(model_uid=prod_model_uid,
                                   deployment_uid=prod_deployment_uid,
                                   space_uid=WML_PROD_SPACE_ID,
                                   label_column=label_column,
                                   training_data_reference=cos_training_data,
                                   test_data_df=data_df,
                                   favorable_classes=['No Risk'],
                                   unfavorable_classes=['Risk'],
                                   quality_config=quality_config,
                                   fairness_config=fairness_config,
                                   drift_v2_config=drift_v2_config,
                                   prediction_column="prediction",
                                   probability_column="probability",
                                   feature_columns=feature_columns,
                                   categorical_columns=categorical_columns
                                   )

## The following cell execute evaluate function which automatically creates datamart , service provider , subscription and enables monitors - fairness , quality, drift v2. 

In [57]:
result = prod_ai_client.model.evaluate(model, engine_credentials=WML_CREDENTIALS, db_credentials=db_credentials,
                                       operational_space_id=operational_space_id)
subscription_id = result[1]

Setting up Datamart...
Using existing datamart
Setting up Service Binding...
Using existing binding a0ec97b4-4414-410c-980b-7b11828a01eb
Validating model...
Existence check for deployment finished. Deployment is Scikit German Risk Model WML V4 production Model
Existence check for Model finished.
Finished reading training data...
Finished inferring model details
Creating Subscription...



 Waiting for end of deleting subscription 756c8a29-cd57-49fb-8cbc-3ccb25257d39 




finished

---------------------------------------------
 Successfully finished deleting subscription 
---------------------------------------------


Deleted existing subscription for Scikit German Risk Model WML V4 production Model



 Waiting for end of adding subscription d821adac-5eb1-40f5-948d-e47edc7db1a2 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------


Subscription created successfully. Id d821adac-5eb1-40f5-948d-e4

## Storing payload data

In [58]:
from ibm_watson_openscale.supporting_classes.enums import DataSetTypes, TargetTypes
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
import time
import uuid

# Extracting some data from dataset 
payload_dataset = data_df.iloc[150:200].copy().drop(columns=[label_column])
fields = list(payload_dataset.columns)
values = payload_dataset.values.tolist()
payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
# Scoring payload data, it automatically stores in payload table
scoring_response = model.add_payload_logging_record(scoring_payload=payload_scoring)

print("Waiting 30 seconds for payload records to reach datamart")
time.sleep(30)
payload_id = prod_ai_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING,
                                           target_target_id=subscription_id,
                                           target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
pl_records_count = prod_ai_client.data_sets.get_records_count(payload_id)
print("Total payload logging records {}".format(pl_records_count))

# If scored data is not reached pl table, explicitly store in pl table 
if pl_records_count == 0:
    print("Payload logging did not happen, performing explicit payload logging.")
    prod_ai_client.data_sets.store_records(data_set_id=payload_id, request_body=[PayloadRecord(
        scoring_id=str(uuid.uuid4()),
        request=payload_scoring,
        response=scoring_response,
        response_time=460
    )])
    time.sleep(5)
    pl_records_count = prod_ai_client.data_sets.get_records_count(payload_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

Waiting 30 seconds for payload records to reach datamart
Total payload logging records 51


## Storing feedback data

In [59]:
import json
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/additional_feedback_data_v2.json

with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)

feedback_dataset_id = prod_ai_client.data_sets.list(type=DataSetTypes.FEEDBACK,
                                                    target_target_id=subscription_id,
                                                    target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[
    0].metadata.id

prod_ai_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data,
                                       background_mode=False)

feedback_count = prod_ai_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)
print("Number of records in the Feedback table: {}".format(feedback_count))

--2024-10-30 12:10:15--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/additional_feedback_data_v2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50890 (50K) [text/plain]
Saving to: ‘additional_feedback_data_v2.json’

additional_feedback 100%[===================>]  49.70K  --.-KB/s    in 0.02s   

2024-10-30 12:10:15 (2.36 MB/s) - ‘additional_feedback_data_v2.json’ saved [50890/50890]




 Waiting for end of storing records with request id: 65f9acf4-2a92-4f71-9482-332303c62da2 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------


Number of records in the Feedback table: 98


## Evaluating configure monitors

In [60]:
prod_ai_client.model.run_evaluation()

Triggering Mrm risk evaluations request...
Grabbing Mrm risk evaluations results...
Checking for status of risk evaluation run
06:40:34 running
06:40:46 running
06:41:00 running
06:41:13 running
06:41:25 running
06:41:37 running
06:41:50 running
06:42:07 running
06:42:25 running
06:42:37 running
06:42:50 running
06:43:02 running
06:43:14 running
06:43:27 running
06:43:40 running
06:43:57 running
06:44:09 running
06:44:21 running
06:44:33 running
06:44:45 running
06:44:57 running
06:45:09 running
06:45:21 running
06:45:34 running
06:45:46 running
06:45:58 running
06:46:10 running
06:46:23 running
06:46:36 running
06:46:51 running
06:47:04 running
06:47:16 running
Mrm risk evaluation finished successfully


## Get monitor instance details for the monitors configured
 The following cell will collect monitor_instance_id details for particular subscription_Id from ibm_watson_opscale SDK.

In [61]:
monitor_instance_ids = {}
monitor_instance_list = prod_ai_client.monitor_instances.list(target_target_id=subscription_id).result
for instance in monitor_instance_list.monitor_instances:
    monitor_instance_ids.update({instance.entity.monitor_definition_id: instance.metadata.id})
monitor_instance_ids

{'fairness': '91d62032-3224-4871-8e3b-d008bc97dc9e',
 'model_health': 'dc16e481-931a-46c8-912a-1abcee638737',
 'quality': '6e1d6e35-7d3b-4b12-aa19-5da7a3b7b097',
 'mrm': '6308846b-1f8b-423f-9b32-cf2ac9a75c98',
 'drift_v2': '8401eaba-e54a-4f06-99b3-44b1d7f0da17',
 'explainability': '9683b7df-ed27-4541-b150-19e35e387ce6',
 'performance': '858d9bf1-77b8-4d97-a753-d53ddee2def4'}

## Fairness monitor details

In [62]:
prod_ai_client.monitor_instances.show_metrics(monitor_instance_id=monitor_instance_ids['fairness'])

2024-10-30 06:40:41.549039+00:00,fairness_value,2f6d48ce-ee24-4ab2-ba8c-43620b21754d,96.774,95.0,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,91d62032-3224-4871-8e3b-d008bc97dc9e,2e45b790-fef6-4c13-a370-36b2f0ed4780,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:41.549039+00:00,fairness_value,2f6d48ce-ee24-4ab2-ba8c-43620b21754d,100.0,95.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,91d62032-3224-4871-8e3b-d008bc97dc9e,2e45b790-fef6-4c13-a370-36b2f0ed4780,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2


## Quality monitor details

In [63]:
prod_ai_client.monitor_instances.show_metrics(monitor_instance_id=monitor_instance_ids['quality'])

2024-10-30 06:40:32.464000+00:00,true_positive_rate,c4fb3955-e4cb-412b-8c92-827e5105e59c,0.5757575757575758,None,None,['model_type:original'],quality,6e1d6e35-7d3b-4b12-aa19-5da7a3b7b097,47833687-2655-4ac2-81a9-04c62d04edef,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:32.464000+00:00,area_under_roc,c4fb3955-e4cb-412b-8c92-827e5105e59c,0.7186480186480186,0.8,None,['model_type:original'],quality,6e1d6e35-7d3b-4b12-aa19-5da7a3b7b097,47833687-2655-4ac2-81a9-04c62d04edef,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:32.464000+00:00,precision,c4fb3955-e4cb-412b-8c92-827e5105e59c,0.6785714285714286,None,None,['model_type:original'],quality,6e1d6e35-7d3b-4b12-aa19-5da7a3b7b097,47833687-2655-4ac2-81a9-04c62d04edef,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:32.464000+00:00,matthews_correlation_coefficient,c4fb3955-e4cb-412b-8c92-827e5105e59c,0.4574684571392756,None,None,['model_type:original'],quality,6e1d6e35-7d3b-4b12-aa19-5da7a3b7b097,47833687-2655-4ac2-81a9-04c62d04edef,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:32.464000+00:00,f1_measure,c4fb3955-e4cb-412b-8c92-827e5105e59c,0.6229508196721312,None,None,['model_type:original'],quality,6e1d6e35-7d3b-4b12-aa19-5da7a3b7b097,47833687-2655-4ac2-81a9-04c62d04edef,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:32.464000+00:00,accuracy,c4fb3955-e4cb-412b-8c92-827e5105e59c,0.7653061224489796,None,None,['model_type:original'],quality,6e1d6e35-7d3b-4b12-aa19-5da7a3b7b097,47833687-2655-4ac2-81a9-04c62d04edef,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:32.464000+00:00,label_skew,c4fb3955-e4cb-412b-8c92-827e5105e59c,0.690933627340049,None,None,['model_type:original'],quality,6e1d6e35-7d3b-4b12-aa19-5da7a3b7b097,47833687-2655-4ac2-81a9-04c62d04edef,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:32.464000+00:00,gini_coefficient,c4fb3955-e4cb-412b-8c92-827e5105e59c,0.4372960372960373,None,None,['model_type:original'],quality,6e1d6e35-7d3b-4b12-aa19-5da7a3b7b097,47833687-2655-4ac2-81a9-04c62d04edef,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:32.464000+00:00,log_loss,c4fb3955-e4cb-412b-8c92-827e5105e59c,0.45766200172124494,None,None,['model_type:original'],quality,6e1d6e35-7d3b-4b12-aa19-5da7a3b7b097,47833687-2655-4ac2-81a9-04c62d04edef,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:32.464000+00:00,false_positive_rate,c4fb3955-e4cb-412b-8c92-827e5105e59c,0.13846153846153847,None,None,['model_type:original'],quality,6e1d6e35-7d3b-4b12-aa19-5da7a3b7b097,47833687-2655-4ac2-81a9-04c62d04edef,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2


Note: First 10 records were displayed.


## Drift_v2 monitor details

In [64]:
prod_ai_client.monitor_instances.show_metrics(monitor_instance_id=monitor_instance_ids['drift_v2'])

2024-10-30 06:41:12.922993+00:00,records_processed,7c2d93bc-4543-46ae-9f97-d4c81b86c363,51.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,8401eaba-e54a-4f06-99b3-44b1d7f0da17,795f8520-4448-47b8-9207-f304a59c7991,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:41:12.922993+00:00,confidence_drift_score,7c2d93bc-4543-46ae-9f97-d4c81b86c363,0.2693,0.8,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,8401eaba-e54a-4f06-99b3-44b1d7f0da17,795f8520-4448-47b8-9207-f304a59c7991,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:41:12.922993+00:00,records_processed,7c2d93bc-4543-46ae-9f97-d4c81b86c363,51.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,8401eaba-e54a-4f06-99b3-44b1d7f0da17,795f8520-4448-47b8-9207-f304a59c7991,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:41:12.922993+00:00,confidence_drift_score,7c2d93bc-4543-46ae-9f97-d4c81b86c363,0.27,0.8,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,8401eaba-e54a-4f06-99b3-44b1d7f0da17,795f8520-4448-47b8-9207-f304a59c7991,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:41:25.996554+00:00,records_processed,78df6505-fe17-448f-aad4-6e990b7daebf,51.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,8401eaba-e54a-4f06-99b3-44b1d7f0da17,795f8520-4448-47b8-9207-f304a59c7991,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:41:25.996554+00:00,confidence_drift_score,78df6505-fe17-448f-aad4-6e990b7daebf,0.1547,0.8,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,8401eaba-e54a-4f06-99b3-44b1d7f0da17,795f8520-4448-47b8-9207-f304a59c7991,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:41:25.996554+00:00,records_processed,78df6505-fe17-448f-aad4-6e990b7daebf,51.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,8401eaba-e54a-4f06-99b3-44b1d7f0da17,795f8520-4448-47b8-9207-f304a59c7991,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:41:25.996554+00:00,confidence_drift_score,78df6505-fe17-448f-aad4-6e990b7daebf,0.1561,0.8,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,8401eaba-e54a-4f06-99b3-44b1d7f0da17,795f8520-4448-47b8-9207-f304a59c7991,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:41:29.492756+00:00,records_processed,ef9c1408-aeca-4010-b5f9-83bb9e1dcb4c,51.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,8401eaba-e54a-4f06-99b3-44b1d7f0da17,795f8520-4448-47b8-9207-f304a59c7991,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:41:29.492756+00:00,prediction_drift_score,ef9c1408-aeca-4010-b5f9-83bb9e1dcb4c,0.046,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,8401eaba-e54a-4f06-99b3-44b1d7f0da17,795f8520-4448-47b8-9207-f304a59c7991,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2


Note: First 10 records were displayed.


## Model_health monitor details

In [65]:
prod_ai_client.monitor_instances.show_metrics(monitor_instance_id=monitor_instance_ids['model_health'])

2024-10-30 06:40:36.024854+00:00,total_records,907c1b71-671a-4359-acda-7605dbde3d31,50.0,None,None,[],model_health,dc16e481-931a-46c8-912a-1abcee638737,28e4c418-f84c-48cb-bb04-3b791f0b22c4,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:36.024854+00:00,maximum_payload_size,907c1b71-671a-4359-acda-7605dbde3d31,7.8125,None,None,[],model_health,dc16e481-931a-46c8-912a-1abcee638737,28e4c418-f84c-48cb-bb04-3b791f0b22c4,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:36.024854+00:00,maximum_record_throughput,907c1b71-671a-4359-acda-7605dbde3d31,7796.104,None,None,[],model_health,dc16e481-931a-46c8-912a-1abcee638737,28e4c418-f84c-48cb-bb04-3b791f0b22c4,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:36.024854+00:00,median_api_throughput,907c1b71-671a-4359-acda-7605dbde3d31,155.922,None,None,[],model_health,dc16e481-931a-46c8-912a-1abcee638737,28e4c418-f84c-48cb-bb04-3b791f0b22c4,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:36.024854+00:00,minimum_api_throughput,907c1b71-671a-4359-acda-7605dbde3d31,155.922,None,None,[],model_health,dc16e481-931a-46c8-912a-1abcee638737,28e4c418-f84c-48cb-bb04-3b791f0b22c4,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:36.024854+00:00,median_record_latency,907c1b71-671a-4359-acda-7605dbde3d31,0.12826919555664062,None,None,[],model_health,dc16e481-931a-46c8-912a-1abcee638737,28e4c418-f84c-48cb-bb04-3b791f0b22c4,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:36.024854+00:00,average_payload_size,907c1b71-671a-4359-acda-7605dbde3d31,7.8125,None,None,[],model_health,dc16e481-931a-46c8-912a-1abcee638737,28e4c418-f84c-48cb-bb04-3b791f0b22c4,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:36.024854+00:00,users,907c1b71-671a-4359-acda-7605dbde3d31,1.0,None,None,[],model_health,dc16e481-931a-46c8-912a-1abcee638737,28e4c418-f84c-48cb-bb04-3b791f0b22c4,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:36.024854+00:00,maximum_api_latency,907c1b71-671a-4359-acda-7605dbde3d31,6.413459777832031,None,None,[],model_health,dc16e481-931a-46c8-912a-1abcee638737,28e4c418-f84c-48cb-bb04-3b791f0b22c4,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2
2024-10-30 06:40:36.024854+00:00,maximum_api_throughput,907c1b71-671a-4359-acda-7605dbde3d31,155.922,None,None,[],model_health,dc16e481-931a-46c8-912a-1abcee638737,28e4c418-f84c-48cb-bb04-3b791f0b22c4,subscription,d821adac-5eb1-40f5-948d-e47edc7db1a2


Note: First 10 records were displayed.


## User can navigate to see an evaluated result in the UI
### Configure dashboard url

In [66]:
prod_integrated_url = prod_ai_client.service_url[0:prod_ai_client.service_url.index("/openscale")].replace(
    "api.aiopenscale",
    "dataplatform") + f"/ml-runtime/deployments/{prod_deployment_uid}/evaluations?space_id={WML_PROD_SPACE_ID}"
print(f"DashBoard URL: \nIntegrated Experience url: {prod_integrated_url}")

DashBoard URL: 
Integrated Experience url: https://dataplatform.cloud.ibm.com/ml-runtime/deployments/30c56a1f-ad47-4de2-95b0-41ce84447f76/evaluations?space_id=d8b93cb6-2caa-49f4-b08e-6a6b6521adee


# Dashboard url of both production and pre-production model

In [67]:
print(f"Pre_production model dashBoard URL: {pre_prod_integrated_url}")
print(f"Production model dashBoard URL: {prod_integrated_url}")

Pre_production model dashBoard URL: https://dataplatform.cloud.ibm.com/ml-runtime/deployments/c44269c8-7e2c-4ab5-acd8-1ea36a046b28/evaluations?space_id=fc5257ff-f3a2-4fe9-a8d8-0f759e1f8007
Production model dashBoard URL: https://dataplatform.cloud.ibm.com/ml-runtime/deployments/30c56a1f-ad47-4de2-95b0-41ce84447f76/evaluations?space_id=d8b93cb6-2caa-49f4-b08e-6a6b6521adee
